In [1]:
import numpy as np
import numba
import matplotlib.pyplot as plt
from pprint import pprint
from IPython.display import Math

In [2]:
basis_vectors = {
    'H': np.array([1, 0]),
    'V': np.array([0, 1]),
}

basis_vectors.update({
    'D': (1/np.sqrt(2)) * (basis_vectors['H'] + basis_vectors['V']),
    'A': (1/np.sqrt(2)) * (basis_vectors['H'] - basis_vectors['V']),
    'R': (1/np.sqrt(2)) * (basis_vectors['H'] + (basis_vectors['V'] * 1j)),
    'L': (1/np.sqrt(2)) * (basis_vectors['H'] - (basis_vectors['V'] * 1j))
    })

In [3]:
def outer(a,b):
    # source: (https://stackoverflow.com/a/25818894)
    # custom 'outer' for this issue
    # a,b must be np.char.array for '+' to be defined
    return a.ravel()[:, np.newaxis]+b.ravel()[np.newaxis,:]

In [4]:
def rotate(angle, deg=True):
    if deg:
        angle = np.deg2rad(angle)
   
    c, s = np.cos(angle), np.sin(angle),
    return np.array([[c, s], [-s, c]])

In [5]:
def vector_from_angle(angle, vector):
    a, b = rotate(angle)
    return np.array([np.sum(vector*a), np.sum(vector*b)])

In [6]:
vector_from_angle(22.5, basis_vectors['H'])

array([ 0.92387953, -0.38268343])

In [7]:
class chsh_measurement():
    def __init__(self, key, angle, prime=False, perp=False):
        self.prime = prime
        self.perp = perp
        self._angle = angle
        self.angle = angle
        
        if isinstance(self.prime, str):
            if self.prime.lower() == 'true':
                self.prime = True
        if isinstance(self.perp, str):
            if self.perp.lower() == 'true':
                self.perp = True
                
        if self.prime:
            self.angle += 45
        
        if self.perp:
            self.angle += 90
                
        self.key_base = key
        self.key = key
        self.vector = vector_from_angle(self.angle, basis_vectors['H'])
    
    @property
    def key(self):
        return self._key
    
    @key.setter
    def key(self, key_str):
            
        if self.prime is True:
            _prime = '^{\prime}'
        else:
            _prime = ''
        
        if self.perp is True:
            _perp = '_{\perp}'
        else:
            _perp = ''
            
    
        self._key = f'\\{key_str}{_prime}{_perp}'

In [8]:
class chsh_expectation():
    def __init__(self, a, b):
        self.a = a
        self.b = b
        
        self.c = chsh_measurement(self.a.key_base, self.a._angle, self.a.prime, perp=True)
        self.d = chsh_measurement(self.b.key_base, self.b._angle, self.b.prime, perp=True)
        
        
    def probability(self, a, b, state_vector):
        out_str = f'{a.key},{b.key}'
        
        out_val = np.abs(
                       np.dot(
                           np.kron(a.vector, b.vector).conj(),
                           state_vector
                              )
                       )
        
        
        return {out_str: out_val}
    
    
    def probabilites(self, state_vector):
        
        p = {}
        p.update(self.probability(self.a, self.b, state_vector))
        p.update(self.probability(self.c, self.b, state_vector))
        p.update(self.probability(self.a, self.d, state_vector))
        p.update(self.probability(self.c, self.d, state_vector))
        
        return p
    
    def expectation(self, state_vector):
        return np.sum(np.array(list(self.probabilites(state_vector).values())) * np.array([1, -1, -1, 1]))

In [9]:
basis_labels = np.char.array(['h', 'v'])
phi, psi = outer(basis_labels, basis_labels).reshape([int(basis_labels.size*2)])[np.array([0, 3, 1, 2])].reshape([2, 2])
phi, psi

(chararray(['hh', 'vv'], dtype='<U2'), chararray(['hv', 'vh'], dtype='<U2'))

In [10]:
state_vecs = {
    'phim': np.subtract(*[np.kron(*[basis_vectors[p.upper()] for p in pair]) for pair in phi]) / np.sqrt(2),
    'phip': np.add(*[np.kron(*[basis_vectors[p.upper()] for p in pair]) for pair in phi]) / np.sqrt(2),
    'psim': np.subtract(*[np.kron(*[basis_vectors[p.upper()] for p in pair]) for pair in psi]) / np.sqrt(2),
    'psip': np.add(*[np.kron(*[basis_vectors[p.upper()] for p in pair]) for pair in psi]) / np.sqrt(2),
}

In [11]:
a = chsh_measurement('alpha', 0, prime=False)
b = chsh_measurement('beta', -22.5, prime=False)
ap = chsh_measurement('alpha', 0, prime=True)
bp = chsh_measurement('beta', -22.5, prime=True)

expect = {
        'ab': chsh_expectation(a, b),
        'apb': chsh_expectation(ap, b),
        'abp': chsh_expectation(a, bp),
        'apbp': chsh_expectation(ap, bp)
        }

angle_pairs = [(e, [expect[e].a.angle, expect[e].b.angle, expect[e].c.angle, expect[e].d.angle]) for e in expect.keys()]

pprint(angle_pairs)

[('ab', [0, -22.5, 90, 67.5]),
 ('apb', [45, -22.5, 135, 67.5]),
 ('abp', [0, 22.5, 90, 112.5]),
 ('apbp', [45, 22.5, 135, 112.5])]


In [12]:
expect = {
        f'{a.key},{b.key}': chsh_expectation(a, b),
        f'{a.key},{bp.key}': chsh_expectation(a, bp),
        f'{ap.key},{b.key}': chsh_expectation(ap, b),
        f'{ap.key},{bp.key}': chsh_expectation(ap, bp),
        }

In [13]:
display(Math('\\newline '.join([f'E\\left({key}\\right):{expect[key].expectation(state_vecs["phim"]):.3g}' for key in expect.keys()])))

<IPython.core.display.Math object>

In [14]:
display(Math('\\newline '.join([f'E\\left({key}\\right):{expect[key].expectation(state_vecs["phip"]):.3g}' for key in expect.keys()])))

<IPython.core.display.Math object>

In [15]:
display(Math('\\newline '.join([f'E\\left({key}\\right):{expect[key].expectation(state_vecs["psim"]):.3g}' for key in expect.keys()])))

<IPython.core.display.Math object>

In [16]:
display(Math('\\newline '.join([f'E\\left({key}\\right):{expect[key].expectation(state_vecs["psip"]):.3g}' for key in expect.keys()])))

<IPython.core.display.Math object>

In [17]:
display(Math('\\newline '.join([f'p\\left({k}\\right):{v:.3g}' for k,v in expect[f'{a.key},{b.key}'].probabilites(state_vecs["phip"]).items()])))

<IPython.core.display.Math object>

In [18]:
sigma_x = np.array([0, 1, 1, 0]).reshape([2, 2])
sigma_y = np.array([0, -1j, 1j, 0]).reshape([2, 2])
sigma_z = np.array([1, 0, 0, -1]).reshape([2, 2])

In [19]:
CHSH_observables = {
    'Q': sigma_z,
    'R': sigma_x,
    'S': (-sigma_z - sigma_x) / np.sqrt(2),
    'T': (sigma_z - sigma_x) / np.sqrt(2)
}

In [20]:
CHSH_observables['S']

array([[-0.70710678, -0.70710678],
       [-0.70710678,  0.70710678]])

In [21]:
def expectation_value(state_vec, observation):
    return np.dot(state_vec,
                  np.dot(observation, state_vec)
                 )

In [22]:
def observable(q1, q2):
    return np.kron(q1, q2)

In [23]:
qs = observable(CHSH_observables['Q'], CHSH_observables['S'])
rs = observable(CHSH_observables['R'], CHSH_observables['S'])
rt = observable(CHSH_observables['R'], CHSH_observables['T'])
qt = observable(CHSH_observables['Q'], CHSH_observables['T'])

In [24]:
qs

array([[-0.70710678, -0.70710678, -0.        , -0.        ],
       [-0.70710678,  0.70710678, -0.        ,  0.        ],
       [-0.        , -0.        ,  0.70710678,  0.70710678],
       [-0.        ,  0.        ,  0.70710678, -0.70710678]])

In [25]:
chsh_pairs = {'qs': qs,
              'rs': rs,
              'rt': rt,
              'qt': qt}

In [26]:
all_chsh_combs = outer(np.char.array([f'{sv}_' for sv in state_vecs.keys()]), np.char.array(list(chsh_pairs.keys())))
pprint(all_chsh_combs)

chararray([['phim_qs', 'phim_rs', 'phim_rt', 'phim_qt'],
           ['phip_qs', 'phip_rs', 'phip_rt', 'phip_qt'],
           ['psim_qs', 'psim_rs', 'psim_rt', 'psim_qt'],
           ['psip_qs', 'psip_rs', 'psip_rt', 'psip_qt']], dtype='<U7')


In [27]:
chsh_expectations = np.zeros(all_chsh_combs.shape)
for i, row in enumerate(all_chsh_combs):
    for j, setting in enumerate(row):
        sv, obs = all_chsh_combs[i][j].split('_')
        chsh_expectations[i][j] = expectation_value(state_vecs[sv], chsh_pairs[obs])
        
chsh_expectations

array([[-0.70710678,  0.70710678,  0.70710678,  0.70710678],
       [-0.70710678, -0.70710678, -0.70710678,  0.70710678],
       [ 0.70710678,  0.70710678,  0.70710678, -0.70710678],
       [ 0.70710678, -0.70710678, -0.70710678, -0.70710678]])

In [28]:
def magnitude(vec):
    return np.sqrt(np.sum([v**2 for v in vec]))

In [29]:
def angle_between_vecs(v1, v2, deg=True):
    angle = np.arccos(np.dot(v1, v2) / (magnitude(v1) * magnitude(v2)))
    
    if deg:
        return np.rad2deg(angle)
    else:
        return angle

In [30]:
v, A = np.linalg.eig(CHSH_observables['T'])

for b in 'HVDA':
    ang = angle_between_vecs(basis_vectors[b], A[:,0])
    print(f'{b}: {ang:.5g}')

H: 22.5
V: 112.5
D: 67.5
A: 22.5


In [31]:
v, A = np.linalg.eig(CHSH_observables['S'])

for b in 'HVDA':
    ang = angle_between_vecs(basis_vectors[b], A[:,0])
    print(f'{b}: {ang-90:.5g}')

H: 67.5
V: 22.5
D: 67.5
A: 22.5
